# Introducción a Deep Learning

En este taller se dará una introducción a cómo funcionan las redes neuronales. Utilizaremos ```Keras```, una librería que hace fácil manipular redes neuronales utilizando Tensorflow, la librería más utilizada para Machine Learning.

Corre ```conda install keras``` para instalar Keras.

Implementaremos un multi-layer perceptron (red neuronal) que pueda identificar dígitos escritos a mano. Dado que es un problema muy común, ya existe un dataset de 60,000 imágenes de dígitos, llamada [MNIST](http://yann.lecun.com/exdb/mnist/). Este es un excelente caso para aprender a utilizar redes neuronales en casos reales. [Keras](https://keras.io/) funciona como un API (o librería) de nivel más alto. Esto hará más fácil y rápido el desarrollo.

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

## Comprendiendo la información

Cualquier uso de Deep Learning requiere el manejo de información. Es decir, vamos a entrenar una red neuronal con training data. Antes de poder saltar a este paso, primero se debe entender cómo es el input. No es lo mismo recibir una imagen a color que una imagen en blanco y negro.

Por suerte, Keras ya viene con las utilidades necesarias para descargar MNIST, lo cuál hace más rápido el trabajo. En la siguiente celda descargamos las imágenes y las dividimos en training y testing sets.

In [ ]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
print(x_train.shape)
print(x_test.shape)
x_train[0]

## Preprocesamos la información
Una red neuronal normal no puede recibir matrices como entrada, sólo vectores. Por lo tanto, convertiremos la lista de listas (matriz) en una lista completa.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def show_digit(index):
    label = y_train[index]
    image = x_train[index]
    plt.title('Training data. Label: {}'.format(label))
    plt.imshow(image, cmap='gray_r')
    plt.show()
show_digit(1)

In [ ]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [ ]:
print(x_train.shape)
print(x_test.shape)

In [ ]:
x_train[0]

In [ ]:
y_train[0]

A los vectores les aplicamos [one hot encoding](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f), una técnica importante para que las redes neuronales puedan utilizarlo. 

Usamos https://keras.io/utils/#to_categorical

In [ ]:
# Has el one hot encoding para que 5 se convierta en [0, 0, 0, 0, 1, 0, 0, 0, 0]
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)
y_train[0]

# 5 -> [0 0 0 0 0 1 0 0 0 0]

En nuestro valor de X, tenemos la entrada, la cuál es la representación de la imagen como vector. En nuestro valor de Y, tenemos el dígito (0 a 9). Dado que manejamos one hot encoding, lo representamos como un vector de 10 elementos. Es importante tener esto en cuenta cuando diseñemos nuestra red neuronal. 

## Construimos arquitectura de red neuronal

Para el entrenamiento utilizaremos 60,000 imágenes. Para hacer el testing utilizaremos 10,000 imágenes. ¿Cuál es la diferencia?

El proceso de entrenamiento es el siguiente para cada imagen.

* La imagen pasa por la red neuronal. (Feedforward)
* La red neuronal predice qué número es.
* Como estamos en entrenamiento, tenemos el valor de Y (el dígito verdadero).
* En base a esto, utilizando un proceso llamado backpropagation, la red neuronal adaptará sus pesos. Usamos Gradient Descent en este paso.
* Este proceso lo repetiremos para las 60,000 imágenes. Todo esto ocurre en un epoch. Dependiendo el tipo de problema, queremos que todo nuestro testing set pase varias veces. Para los problemas sencillos, entre 10 y 200 epochs suele ser más que suficiente.

* Así que, si tenemos 60,000 imágenes y queremos entrenar nuestra red neuronal en 10 epochs, estaremos hablando de pasar 600,000 imágenes a la red neuronal. Con razón recién es un área que ha empezado a ganar popularidad en los últimos años.

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(784,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd', #stochastic gradient descent
              metrics=['accuracy'])

## Entrenamos el modelo

In [ ]:
model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=50,
                    verbose=1,
                    validation_data=(x_test, y_test))

## Evaluamos el modelo

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', score[1])